In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from mw import ManifoldWorms
from torch import optim
import gymnasium as gym


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\rubn\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\rubn\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\rubn\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.

In [2]:
1 - torch.acos(torch.Tensor([[ 1.5077, -1.2859, -0.4764, -1.0582]])) / torch.pi * .5


tensor([[   nan,    nan, 0.6710,    nan]])

In [ ]:
channel_size = 1
env_dims = 3
input_size = 4
output_size = 1
n_units = 0
reach = 1


mw = ManifoldWorms(
    channel_size=channel_size,
    env_dims=env_dims,
    input_size=input_size,
    output_size=output_size,
    n_units=n_units,
    reach=reach,
)

optimizer = optim.Adam(mw.parameters(), lr=0.01)

for epoch in range(100):
    outputs, garbage = mw.forward(torch.ones(input_size, channel_size) * .5, True)
    print("Outputs", outputs)
    print("Garbage", garbage)
    loss = F.mse_loss(outputs, torch.ones(output_size, channel_size)) + garbage.sum()
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    print("Loss", loss.item())




Similarities tensor([[-0.0352,  0.6105,  0.5423,  0.7802]], grad_fn=<MmBackward0>)
influences tensor([[-0.0224,  0.4180,  0.3649,  0.5698]], grad_fn=<RsubBackward1>)
data tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000]])
outputs tensor([[0.6652]], grad_fn=<SliceBackward0>)
garbage tensor([1.3348], grad_fn=<SumBackward1>)
1.4469743967056274
Similarities tensor([[-0.0390,  0.6147,  0.5373,  0.7841, -0.0232,  0.6305,  0.5531,  0.7999]],
       grad_fn=<MmBackward0>)
influences tensor([[-0.0248,  0.4215,  0.3611,  0.5738, -0.0148,  0.4343,  0.3731,  0.5903]],
       grad_fn=<RsubBackward1>)
data tensor([[0.0511],
        [0.0291],
        [0.0318],
        [0.0215],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<CatBackward0>)
outputs tensor([[0.7263]], grad_fn=<SliceBackward0>)
garbage tensor([1.4072], grad_fn=<SumBackward1>)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 3]] is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [46]:
env = gym.make(
    "LunarLander-v3",
    continuous=False,
    gravity=-9.8,
    enable_wind=False,
    wind_power=15.0,
    turbulence_power=1.5,
)
action_space = int(env.action_space.n)
obs_space = env.observation_space.shape[0]

In [45]:
def to_normalized_tensor(obs):
    _RANGE = torch.Tensor(env.observation_space.high - env.observation_space.low)
    return torch.Tensor(obs) / _RANGE

In [ ]:
torc

In [186]:
torch.acos(torch.Tensor([0.8]))

tensor([0.6435])

In [184]:
torch.pi * .5

1.5707963267948966

In [30]:
obs, _ = env.reset(seed=42)

In [4]:
mw = ManifoldWorms(
    channel_size=1,
    env_dims=5,
    input_size=obs_space,
    output_size=action_space,
    n_units=16,
    reach=1,
)
optimizer = optim.Adam(mw.parameters(), lr=0.01)

In [ ]:
obs, _ = env.reset(seed=42)

episode_over = False
while not episode_over:

    obs = to_normalized_tensor(obs)
    outputs, garbage = mw.forward(obs, True)
    outputs = F.softmax(outputs, dim=0)
    action = torch.multinomial(outputs, 1)

    obs, reward, terminated, truncated, _ = env.step(action)
    episode_over = terminated or truncated

env.close()